In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import matplotlib.pyplot as plt
import torch.nn.init as init
import time
import random
import pandas as pd
import openpyxl

In [14]:
N_x = 81
N_y = 81

x_l = 0
y_b = 0
x_r = 1
y_t = 1

N_t = 250000
dt = 0.00001

f_lim = 1

# material params:- RT-35
k_therm = 0.2
L = 160000
Cp = 2000
rho = 800 
T_solidus = 305
T_liquidus = 313

### Assumed boundary conditions ####
q = 450

#### Normalising coeffs #####
T_change = 0.35
# T_change = 0.25
delta_x = 0.01
delta_y = 0.01
# delta_T = (1 - T_change)*(T_liquidus - T_solidus)/T_change
delta_T = (T_liquidus - T_solidus)/T_change

#### k1 ###
k1 = k_therm/(rho*Cp*delta_x**2)*300

### k2 ###
k2 = k_therm/(rho*(Cp+L/(T_liquidus - T_solidus))*delta_x**2)*300

#### q_bnorm #####
q_norm = q*delta_x/(k_therm*delta_T)

print('k1 = ',k1,' k2 = ', k2, ' q_norm = ', q_norm)

cfd_check = k1*dt/( (x_r-x_l)/(N_x-1) )**2
print('cfd_check = ', cfd_check)

k1 =  0.375  k2 =  0.03409090909090909  q_norm =  0.9843749999999999
cfd_check =  0.023999999999999997


In [15]:
# X, Y matrix
X = np.linspace(x_l,x_r,N_x)
dx = X[1] - X[0]
X = np.tile(X,N_y)
X = np.reshape(X, (N_x,N_y))

Y = np.linspace(y_b,y_t,N_y)
dy = Y[1] - Y[0]
Y = np.repeat(Y,N_x)
Y = np.reshape(Y, (N_x,N_y))

# liquid fraction, battery and initial temperature matrix
f = np.zeros((N_x,N_y))
T = np.zeros((N_x,N_y))

# for i in range(N_y):
#     for j in range(N_x):

#         if i==0 or j==0:
#             T[i][j] = 1
#             f[i][j] = 1

In [16]:
aN = np.zeros((N_x,N_y))
aS = np.zeros((N_x,N_y))
aE = np.zeros((N_x,N_y))
aW = np.zeros((N_x,N_y))
aP = np.zeros((N_x,N_y))
flag = np.zeros((N_x,N_y))

for i in range(N_x):
    for j in range(N_y):
        
        if Y[i][j]!=y_t:
            aN[i][j] = k2*dt/dy**2
            
        if Y[i][j]!=y_b:
            aS[i][j] = k2*dt/dy**2
        
        if X[i][j]!=x_l:
            aW[i][j] = k2*dt/dx**2
            
        if X[i][j]!=x_r:
            aE[i][j] = k2*dt/dx**2
            
        aP[i][j] = 1 - (aN[i][j] + aS[i][j] + aW[i][j] + aE[i][j])

In [17]:
aP

array([[0.99563636, 0.99345455, 0.99345455, ..., 0.99345455, 0.99345455,
        0.99563636],
       [0.99345455, 0.99127273, 0.99127273, ..., 0.99127273, 0.99127273,
        0.99345455],
       [0.99345455, 0.99127273, 0.99127273, ..., 0.99127273, 0.99127273,
        0.99345455],
       ...,
       [0.99345455, 0.99127273, 0.99127273, ..., 0.99127273, 0.99127273,
        0.99345455],
       [0.99345455, 0.99127273, 0.99127273, ..., 0.99127273, 0.99127273,
        0.99345455],
       [0.99563636, 0.99345455, 0.99345455, ..., 0.99345455, 0.99345455,
        0.99563636]])

In [18]:
storage_interval = 10000

T_str = []
T_str.append(np.copy(T))
T_new = np.copy(T)

f_str = []
f_str.append(np.copy(f))
f_new = np.copy(f)      

for t in range(N_t):
    if t%10000==0:
        print('t = ',(t+1)*dt)
    
    T = np.copy(T_new)
    f = np.copy(f_new)

    # Temperature Update
    for i in range(N_y):
        for j in range(N_x):

            if ( X[i][j]!=x_l and X[i][j]!=x_r and Y[i][j]!=y_t and Y[i][j]!=y_b ):
                
                T_new[i][j] = ( aN[i][j]*T[i+1][j] + aS[i][j]*T[i-1][j] + 
                                   aW[i][j]*T[i][j-1] + aE[i][j]*T[i][j+1] + aP[i][j]*T[i][j] )
                
                f_new[i][j] = T_new[i][j]/T_change

    # LEFT BOUNDARY
    for i in range(1, N_y - 1):
        T_new[i][0] = q_norm*dx + T_new[i][1]
        f_new[i][0] = T_new[i][0]/T_change

    # BOTTOM BOUNDARY
    for j in range(1, N_x - 1):
        T_new[0][j] = q_norm*dy + T_new[1][j]
        f_new[0][j] = T_new[0][j]/T_change

    # TOP BOUNDARY
    for i in [N_y-1]:
        for j in range(N_x):
            T_new[i][j] = T_new[i-1][j]
            f_new[i][j] = T_new[i][j]/T_change
    # RIGHT BOUNDARY
    for j in [N_x-1]:
        for i in range(N_y):
            T_new[i][j] = T_new[i][j-1]
            f_new[i][j] = T_new[i][j]/T_change

    # CORNER POINT
    T_new[0][0] = q_norm*dx + T_new[1][0] 
    f_new[0][0] = T_new[0][0]/T_change

    ######### Liquid Fraction Update #########
    for i in range(N_y):
        for j in range(N_x):
        
            if T_new[i][j]>=T_change or f_new[i][j]>=1:

                f_new[i][j] = 1

                if Y[i][j]!=y_t:
                    aN[i][j] = k1*dt/dy**2

                if Y[i][j]!=y_b:
                    aS[i][j] = k1*dt/dy**2

                if X[i][j]!=x_l:
                    aW[i][j] = k1*dt/dx**2

                if X[i][j]!=x_r:
                    aE[i][j] = k1*dt/dx**2

                aP[i][j] = 1 - (aN[i][j] + aS[i][j] + aW[i][j] + aE[i][j])

    if (t+1)%storage_interval==0:
        T_str.append(np.copy(T_new))
        f_str.append(np.copy(f_new))

t =  1e-05
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_10148\319715882.py", line -1, in <cell line: 11>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py", line 1982, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
    return FormattedTB.structured_traceback(
  File "C:\Users\USER\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\ultratb.py", line 1012, in structured_traceback
    return VerboseTB.structur

In [ ]:
t

In [ ]:
# Temperature plots
k = 0
for i in range(len(T_str)):
    
    plt.figure()
    plt.figure(figsize=(5,4))
    sc = plt.scatter(X, Y, c = T_str[i], cmap=plt.cm.jet)
    plt.colorbar(sc)
    plt.clim(0, 0.6)
    plt.title('t = '+ str(round(i*storage_interval*dt, 2)) +', Temperature plot')

plt.show()

In [ ]:
# Liquid Fraction plots
k = 0
for i in range(len(T_str)):
    
    plt.figure()
    plt.figure(figsize=(5,4))
    sc = plt.scatter(X, Y, c = T_str[i]>0.35, cmap=plt.cm.jet)
    plt.colorbar(sc)
    plt.clim(0, 1)
    plt.title('t = '+  str(round(i*storage_interval*dt, 2)) +', Liquid Fraction plot')

plt.show()

In [ ]:
T_str[-1]

In [ ]:
# Store arrays in excel file

workbook = openpyxl.Workbook()
i = 0
for arr in T_str:
    # Create two sheets in the workbook
    sheet = workbook.create_sheet(title="t = "+ str(round(i*storage_interval*dt, 2)))
    # Populate the first sheet with data from the first array
    array = arr.tolist()
    for row in array:
        sheet.append(row)
    i = i+1

# Save the workbook to the specified file
file_name = "2d_2side_heating.xlsx"
workbook.save(file_name)
print(f"Data saved to {file_name}")

In [ ]:
# Data for creating graphs
x = X[0]
T = []
for i in range(25):
    T.append(T_str[-1][i][i])
# T = T_str[-1][16]
df = pd.DataFrame(list(zip(x, T)), columns = ['x', 'Temperature'])
df
df.to_excel("cfd_2sideflux_diagonal.xlsx")